## Perguntas

1) Carregar e ler arquivos .csv, xlsx ou de um banco de dados;
2) Avaliar a necessidade da transformação da variável em outra escala (agrupar idade por faixas, por exemplo);
3) Realizar a codificação das variáveis categóricas de acordo com os valores das variáveis (label encoder, one hot encoder ou target encoder);
4) Normalizar as variáveis;
5) Avaliar a necessidade de realizar o balanceamento da variável alvo;
6) Tratar variáveis com alta correlação;
7) Realizar a seleção de variáveis;
8) Aplicar um modelo de regressão ou classificação utilizando uma técnica de hiperparametrização automática;
9) Aplicar uma medida de avaliação do modelo.
10) Gerar uma visualização para o resultado de acordo com os tipos abaixo:
    - Agrupamento: gerar a visualização Scatter;
    - Classificação: gerar a matriz de confusão; 
    - Regressão: gerar a visualização da linha de saída.

### Leitura dos dados

In [ ]:
# Somente rodar se necessário
# Descomente a linha a baixo para rodar!

# %pip install googletrans==4.0.0-rc1

In [ ]:
#IMPORT E INSTALL AS BIBLIOTECAS IMPORTANTES
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps
from googletrans import Translator # Precisa do pacote do pip install

In [ ]:
caminho = 'Data/Bike-day.csv'
df = pd.read_csv(caminho)

### Tratamento dos dados

#### Tradução 

In [ ]:
def traduzir_colunas(df, language='pt'):
    #Dicionario para as palavras que não foram traduzidas ou que foram traduzidas erroneamente 
    erro_ao_traduzir = {
        'dteday': 'Data',
        'mnth': 'Mês',
        'weathersit': 'Clima',
        'temp': 'Temperatura (C°)',
        'hum': 'Umidade',
        'windspeed': 'Velocidade do Vento',
        'registered': 'Registrados',
        'cnt': 'Contador (Casual e Registrados)',
        'atemp': 'Sensação térmica'
    }
    
    # Renomear as colunas
    traducoes = {}
    tradutor = Translator()
    for coluna in df.columns:
        try:
            if coluna in erro_ao_traduzir:
                traducoes[coluna] = erro_ao_traduzir[coluna] # Utilize a tradução personalizada
            else:
                traduzido = tradutor.translate(coluna, src='en', dest=language) # Tradução acontece aqui
                if traduzido and traduzido.text:
                    capslock = traduzido.text.capitalize() # Maiúscula na primeira letra de cada palavra

                    traducoes[coluna] = capslock
                else:
                    print(f"Erro ao traduzir a coluna '{coluna}': tradução retornou None ou texto vazio")
                    traducoes[coluna] = coluna # Manter o nome original em caso de erro
        except Exception as a:
            print(f"Erro ao traduzir a coluna '{coluna}': {a}")
            traducoes[coluna] = coluna # Manter o nome original em caso de erro

    df.columns = [traducoes.get(col, col) for col in df.columns]

In [ ]:
#EXECUTA A FUNÇÃO DE TRADUÇÃO
df_traduzido = df.copy()
traduzir_colunas(df_traduzido)
df_traduzido.head(5)

In [ ]:
#REMOVE O INDEXADOR DA PLANILHA
df_traduzido = df_traduzido.drop(columns=['Instante'])
df_traduzido.head(5)

In [ ]:
#RENOMEANDO OS VALORES NUMERICOS PARA ESTAÇÕES DO ANO (season : season (1:winter, 2:spring, 3:summer, 4:fall)) e Ano  (yr : year (0: 2011, 1:2012))

if 'Temporada' in df_traduzido.columns:
        df_traduzido['Temporada'] = df_traduzido['Temporada'].replace({1: 'Inverno', 2: 'Primavera', 3: 'Verão', 4: 'Outono'})

if 'Ano' in df_traduzido.columns:
        df_traduzido['Ano'] = df_traduzido['Ano'].replace({0: '2011', 1: '2012'})
df_traduzido.head(5)

#### Informações gerais do dataset

In [ ]:
df_traduzido.describe()

In [ ]:
df_traduzido.info()

### Análise Descritiva dos dados

Gerar tabelas sobre o data set: (gráfico de barras, colunas, linha, etc)

- relação mês x aluguel - guzinho
- relação temperatura x aluguel - guzinho

- concorrência de casual x registrado - vitinho
- Qual é o impacto dos dias úteis e feriados no uso de bicicletas? - marcelo

#### Relação season x aluguel - Allan

In [ ]:
# Gerando o gráfico de análise descritiva
contador_temporada_X_Aluguel = df_traduzido.groupby('Temporada')['Contador (Casual e Registrados)'].sum()

plt.figure(figsize=(8, 5))
contador_temporada_X_Aluguel.plot(kind='bar', color='skyblue')
plt.title('Total de Aluguel de Bicicletas por Temporada')
plt.xlabel('Temporada')
plt.ylabel('Total de Aluguel (Contador)')
plt.xticks(rotation=0)
plt.grid(axis='y')

In [ ]:
# Mostrar o gráfico
plt.show()

In [ ]:
# Identificar as top temporadas com o maior número de aluguéis
top_temporadas = contador_temporada_X_Aluguel.sort_values(ascending=False).head(4)
for i, (temporada, total) in enumerate(top_temporadas.items(), start=1):
    print(f'Top {i}: {temporada} com {total} aluguéis')

### Análise de correlação de variáveis - Marcelo

rodar ppscore pra ver no que da

In [ ]:
sns.heatmap(df.corr(), annot = True)

### Modelos de Regressão Linear

- Regressão linear - murilo
- Lasso - murilo
- Elastic Net - vitinho
- Regressão Ridge - vitinho

In [ ]:
print("teste")